In [ ]:
!pip install transformers
!pip install datasets

In [2]:
import pandas as pd 
#change path as necessary
df = pd.read_csv("/content/hi_3500.csv", header=0, names=['review', 'sentiment'])
# df = df.iloc[]
print(df)


                                                 review sentiment
0     गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...  negative
1     ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है  negative
2     यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...  negative
3     अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...  negative
4                       कश्मीर में हो रहा है जल जिहाद ।  negative
...                                                 ...       ...
9071  RT @aajtak: #Mojo बजरंग बली का सबसे विशालकाय र...   neutral
9072  RT @aajtak: कहां अचानक स्कूल में पड़ा मंत्री क...   neutral
9073  RT @aajtak: #MoJo सुनिए कुलभूषण की दर्दभरी आपब...   neutral
9074  RT @aajtak: लव जेहाद के नाम पर क्या उत्तर प्रद...   neutral
9075  कश्मीर में हमारे जवानों से ऐसा सलूक? वीडियो दे...   neutral

[9076 rows x 2 columns]


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], random_state=91)
from sklearn import preprocessing
labeler = preprocessing.LabelEncoder()

df2 = pd.DataFrame(X_train)
df2['label'] = labeler.fit_transform(y_train)
print(df2.head())

                                                 review  label
2378  RT @itsparvezsagar: शैलजा मर्डरः मृतका का पोस्...      1
1450  सेट और कॉस्ट्यूम में कल्पना और बजट की कटौती से...      0
5513  स प्रदर्शन से मुझे और परिपक्व गेंदबाज बनने में...      2
3061  RT @neerajjournalis: वोट के तैयार है काशी और ह...      1
5975                           मेरी यात्रा आनंदप्रद रही      2


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

model_name = "sandeepvarma99/tacl-hindi"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) 
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sandeepvarma99/tacl-hindi and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import torch
from torch.utils.data import Dataset
def encode(examples):
    return tokenizer(examples, truncation=True, padding='max_length', max_length=512)

# Apply the function on your dataframe
df2['input_ids'] = df2['review'].apply(lambda x: encode(x)['input_ids'])

# Convert lists to tensors
df2['input_ids'] = df2['input_ids'].apply(lambda x: torch.tensor(x))

# Create a Dataset object
class HindiSentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids': self.encodings[idx].clone().detach()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HindiSentimentDataset(list(df2['input_ids']), list(df2['label']))


In [6]:
# Define the training arguments
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3, 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.752700
1000,0.441500


TrainOutput(global_step=1278, training_loss=0.5258051434966133, metrics={'train_runtime': 1989.2117, 'train_samples_per_second': 10.266, 'train_steps_per_second': 0.642, 'total_flos': 5373039103423488.0, 'train_loss': 0.5258051434966133, 'epoch': 3.0})

In [7]:
df_test = pd.DataFrame(X_test)
df_test['label'] = labeler.transform(y_test)

# Apply the encoding function on the test data
df_test['input_ids'] = df_test['review'].apply(lambda x: encode(x)['input_ids'])

# Convert lists to tensors
df_test['input_ids'] = df_test['input_ids'].apply(lambda x: torch.tensor(x))

test_dataset = HindiSentimentDataset(list(df_test['input_ids']), list(df_test['label']))


In [8]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Update the trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate the model on the test dataset
eval_results = trainer.evaluate(test_dataset)

# Print the evaluation results
print("Evaluation Results:", eval_results)

# Get the accuracy
accuracy = eval_results['eval_accuracy']
print("Accuracy:", accuracy)


Evaluation Results: {'eval_loss': 0.3662877678871155, 'eval_accuracy': 0.8788012340237991, 'eval_runtime': 68.7576, 'eval_samples_per_second': 33.0, 'eval_steps_per_second': 0.524}
Accuracy: 0.8788012340237991


In [9]:
import pandas as pd

# Generate predictions on the test dataset
logits = trainer.predict(test_dataset).predictions
predictions = np.argmax(logits, axis=-1)

# Create a DataFrame with test dataset ground truth labels, predictions, and reviews
error_analysis_df = pd.DataFrame({
    'review': X_test,
    'ground_truth_label': y_test,
    'predicted_label': labeler.inverse_transform(predictions)
})

# Filter out instances where the model's prediction is incorrect
incorrect_predictions_df = error_analysis_df[error_analysis_df['ground_truth_label'] != error_analysis_df['predicted_label']]

# Display the incorrect predictions
print("Incorrect Predictions:")



Incorrect Predictions:


In [10]:
display(incorrect_predictions_df)

,review,ground_truth_label,predicted_label
117,~चप्पड़गंजू नसीहत दे रहा है की गुरु का सम्मान ...,negative,positive
4836,इस पर चुप्पी तोड़ते हुए रामपाल ने कहा है कि उन...,positive,negative
5632,"नेता नहीं निभाते फर्ज, लेकिन ग्रामीण फिर भी कर...",positive,negative
1628,एक महिला को मुंबईमें नौकरी मिल गई .वह अकेली ही...,positive,negative
885,@nishantchat @sardanarohit You missed the line...,positive,negative
...,...,...,...
3899,उसका इलेक्ट्रो में परिवर्तित होकर अचानक स्पाइड...,positive,negative
6552,जिससे मच्छर जनित बीमारियां भी बढ़ गई।,negative,positive
5834,स दवा से दो दिन में ही मेरी तबीयत बहाल हो गई,positive,negative
5602,यह कांग्रेस महासचिव का नितांत निजी मामला है।,positive,negative


In [ ]:
!pip install checklist

In [65]:
from checklist.pred_wrapper import PredictorWrapper

def custom_predictor_wrapper(predict_fn):
    def wrapped_predict_fn(texts):
        logits = predict_fn(texts)
        probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
        preds = np.argmax(probs, axis=-1)
        confs = np.max(probs, axis=-1)
        return preds, confs

    return wrapped_predict_fn
def model_predict(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

    # Move input tensors to the same device as the model
    inputs = {name: tensor.to(trainer.model.device) for name, tensor in inputs.items()}

    outputs = trainer.model(**inputs)
    logits = outputs.logits
    logits_np = logits.detach().cpu().numpy()

    return logits_np



wrapped_predictor = custom_predictor_wrapper(model_predict)


In [36]:
from checklist.test_suite import TestSuite
from checklist.test_types import MFT, INV, DIR

suite = TestSuite()

In [68]:
from checklist.editor import Editor

# Instantiate the Editor for Korean
editor = Editor(language="ko")

# Create sentences with specific words
positive_words = ['좋은', '매우 좋은', '멋진']
negative_words = ['나쁜', '안 좋은', '실망스러운']

positive_sentences = [f'이 제품은 {word}입니다.' for word in positive_words]
negative_sentences = [f'이 제품은 {word}입니다.' for word in negative_words]

# Define the test data and expected labels
test_data = positive_sentences + negative_sentences
test_labels = ['positive'] * len(positive_sentences) + ['negative'] * len(negative_sentences)

# Create the vocabulary test
vocabulary_test_params = {
    'data': test_data,
    'labels': test_labels,
    'name': 'Vocabulary Test',
    'description': 'Test the model on simple sentences containing specific words.',
    'capability': 'Vocabulary',
}

# Add the test to the suite
suite.add(MFT(**vocabulary_test_params),overwrite=True)


In [69]:
# Run the test suite on the wrapped model
suite.run(wrapped_predictor, verbose=2,overwrite=True)



Running Vocabulary Test
Predicting 6 examples


In [70]:
summary = suite.summary()
print(summary)

Vocabulary

Vocabulary Test
Test cases:      6
Fails (rate):    6 (100.0%)

Example fails:
2 이 제품은 좋은입니다.
----
0 이 제품은 실망스러운입니다.
----
0 이 제품은 나쁜입니다.
----




None
